<a href="https://colab.research.google.com/github/willyptrain/TextAttackResearch/blob/main/1_Introduction_and_Transformations_adapated_to_attack_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The TextAttack ecosystem: search, transformations, and constraints

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/1_Introduction_and_Transformations.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/1_Introduction_and_Transformations.ipynb)

An attack in TextAttack consists of four parts.

### Goal function

The **goal function** determines if the attack is successful or not. One common goal function is **untargeted classification**, where the attack tries to perturb an input to change its classification. 

### Search method
The **search method** explores the space of potential transformations and tries to locate a successful perturbation. Greedy search, beam search, and brute-force search are all examples of search methods.

### Transformation
A **transformation** takes a text input and transforms it, for example replacing words or phrases with similar ones, while trying not to change the meaning. Paraphrase and synonym substitution are two broad classes of transformations.

### Constraints
Finally, **constraints** determine whether or not a given transformation is valid. Transformations don't perfectly preserve syntax or semantics, so additional constraints can increase the probability that these qualities are preserved from the source to adversarial example. There are many types of constraints: overlap constraints that measure edit distance, syntactical  constraints check part-of-speech and grammar errors, and semantic constraints like language models and sentence encoders.

### A custom transformation

This lesson explains how to create a custom transformation. In TextAttack, many transformations involve *word swaps*: they take a word and try and find suitable substitutes. Some attacks focus on replacing characters with neighboring characters to create "typos" (these don't intend to preserve the grammaticality of inputs). Other attacks rely on semantics: they take a word and try to replace it with semantic equivalents.


### Banana word swap 

As an introduction to writing transformations for TextAttack, we're going to try a very simple transformation: one that replaces any given word with the word 'banana'. In TextAttack, there's an abstract `WordSwap` class that handles the heavy lifting of breaking sentences into words and avoiding replacement of stopwords. We can extend `WordSwap` and implement a single method, `_get_replacement_words`, to indicate to replace each word with 'banana'. 🍌

## Fetching latest TextAttack from attack-api branch


In [1]:
!git clone -b attack-api https://github.com/QData/TextAttack.git
!pip install pyarrow --upgrade


Cloning into 'TextAttack'...
remote: Enumerating objects: 17776, done.
remote: Total 17776 (delta 0), reused 0 (delta 0), pack-reused 17776
Receiving objects: 100% (17776/17776), 23.68 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (13328/13328), done.
     |████████████████████████████████| 17.7MB 1.2MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [1]:
%cd TextAttack/
!python setup.py install
%cd ..

/content/TextAttack
running install
running bdist_egg
running egg_info
writing textattack.egg-info/PKG-INFO
writing dependency_links to textattack.egg-info/dependency_links.txt
writing entry points to textattack.egg-info/entry_points.txt
writing requirements to textattack.egg-info/requires.txt
writing top-level names to textattack.egg-info/top_level.txt
writing manifest file 'textattack.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/textattack
copying build/lib/textattack/__init__.py -> build/bdist.linux-x86_64/egg/textattack
copying build/lib/textattack/attacker.py -> build/bdist.linux-x86_64/egg/textattack
creating build/bdist.linux-x86_64/egg/textattack/shared
copying build/lib/textattack/shared/__init__.py -> build/bdist.linux-x86_64/egg/textattack/shared
copying build/lib/textattack/shared/checkpoint.py -> build/bdist.linux-x86_64/egg/text

In [2]:
from textattack.transformations import WordSwap

class BananaWordSwap(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.
        
            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        return ['banana']

/content/TextAttack


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:18<00:00, 25.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpdok64psi.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


/content


### Using our transformation

Now we have the transformation chosen, but we're missing a few other things. To complete the attack, we need to choose the **search method** and **constraints**. And to use the attack, we need a **goal function**, a **model** and a **dataset**. (The goal function indicates the task our model performs – in this case, classification – and the type of attack – in this case, we'll perform an untargeted attack.)

### Creating the goal function, model, and dataset
We are performing an untargeted attack on a classification model, so we'll use the `UntargetedClassification` class. For the model, let's use BERT trained for news classification on the AG News dataset. We've pretrained several models and uploaded them to the [HuggingFace Model Hub](https://huggingface.co/textattack). TextAttack integrates with any model from HuggingFace's Model Hub and any dataset from HuggingFace's `datasets`!

In [3]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-ag-news")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

# Import the dataset
from textattack.datasets import HuggingFaceDataset
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Using custom data configuration default


textattack: Loading datasets dataset ag_news, split test.


Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a. Subsequent calls will reuse this data.


### Creating the attack
Let's keep it simple: let's use a greedy search method, and let's not use any constraints for now. 

In [5]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack

# We're going to use our Banana word swap class as the attack transformation.
transformation = BananaWordSwap() 
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)


Let's print our attack to see all the parameters:

In [6]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


### Using the attack

Let's use our attack to successfully attack 10 samples.

In [32]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker

attacker = Attacker(attack, dataset)
attacker.attack_dataset()
logManager = attacker.attack_log_manager


# results_iterable = attack.attack_dataset(dataset)
# logger = CSVLogger(color_method='html')
# num_successes = 0
# while num_successes < 10:
#     result = next(results_iterable)
#     if isinstance(result, SuccessfulAttackResult):
        # logger.log_attack_result(result)
#         num_successes += 1
#         print(f'{num_successes} of 10 successes complete.')

  0%|          | 0/5 [00:00<?, ?it/s]

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  BananaWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Total] 1 / 0 / 1:  20%|██        | 1/5 [00:00<00:01,  2.01it/s]

--------------------------------------------- Result 1 ---------------------------------------------
Business (100%) --> World (89%)

Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.

Fears for T N banana after banana banana representing banana at Turner   Newall say they are 'banana after talks with stricken parent firm Federal banana.




[Succeeded / Failed / Total] 2 / 0 / 2:  40%|████      | 2/5 [00:03<00:05,  2.00s/it]

...

--------------------------------------------- Result 3 ---------------------------------------------
Sci/tech (100%) --> Business (77%)

Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.

Ky. Company Wins Grant to banana banana (banana) banana - banana company banana by a banana banana at the banana of Louisville won a grant to develop a method of producing better banana, which are short chains of banana banana, the building blocks of banana.




[Succeeded / Failed / Total] 4 / 0 / 4:  80%|████████  | 4/5 [00:06<00:01,  1.54s/it]

--------------------------------------------- Result 4 ---------------------------------------------
Sci/tech (100%) --> World (65%)

Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.

banana Unit Helps banana Wildfires (AP) banana - It's barely dawn when Mike Fitzpatrick banana his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.




[Succeeded / Failed / Total] 5 / 0 / 5: 100%|██████████| 5/5 [00:06<00:00,  1.33s/it]

--------------------------------------------- Result 5 ---------------------------------------------
Sci/tech (100%) --> World (62%)

Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.

Calif. Aims to Limit Farm-Related banana (AP) AP - Southern California's banana agency went after banana of the banana variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 2

### Visualizing attack results

We are logging `AttackResult` objects using a `CSVLogger`. This logger stores all attack results in a dataframe, which we can easily access and display. Since we set `color_method` to `'html'`, the attack results will display their differences, in color, in HTML. Using `IPython` utilities and `pandas`

In [25]:
logManager = attacker.attack_log_manager
logManager.add_output_csv(filename="results.csv", color_method="file")

textattack: Logging to CSV at path results.csv


In [31]:
logger = logManager.loggers[0]
print(logManager.loggers)

[<textattack.loggers.file_logger.FileLogger object at 0x7f524266b8d0>, <textattack.loggers.csv_logger.CSVLogger object at 0x7f5247d2fd30>, <textattack.loggers.csv_logger.CSVLogger object at 0x7f523eb3e390>, <textattack.loggers.csv_logger.CSVLogger object at 0x7f523fe6bd30>]


In [30]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples


from IPython.core.display import display, HTML
print(logger.df)
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

AttributeError: ignored

### Conclusion
We can examine these examples for a good idea of how many words had to be changed to "banana" to change the prediction score from the correct class to another class. The examples without perturbed words were originally misclassified, so they were skipped by the attack. Looks like some examples needed only a couple "banana"s, while others needed up to 17 "banana" substitutions to change the class score. Wow! 🍌

### Bonus: Attacking Custom Samples

We can also attack custom data samples, like these ones I just made up!

In [ ]:
# For AG News, labels are 0: World, 1: Sports, 2: Business, 3: Sci/Tech

custom_dataset = [
    ('Malaria deaths in Africa fall by 5% from last year', 0),
    ('Washington Nationals defeat the Houston Astros to win the World Series', 1),
    ('Exxon Mobil hires a new CEO', 2),
    ('Microsoft invests $1 billion in OpenAI', 3),
]

results_iterable = attack.attack_dataset(custom_dataset)

logger = CSVLogger(color_method='html')

for result in results_iterable:
    logger.log_attack_result(result)
    
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

textattack: CSVLogger exiting without calling flush().


,original_text,perturbed_text
0,Malaria deaths in Africa fall by 5% from last year,Malaria banana in Africa fall by 5% from last year
1,Washington Nationals defeat the Houston Astros to win the World Series,banana banana banana the Houston Astros to win the World Series
2,Exxon Mobil hires a new CEO,banana banana banana a new banana
3,Microsoft invests $1 billion in OpenAI,banana invests $1 billion in OpenAI
